# Downloading the data

In [7]:
import os
import pandas as pd
import numpy as np

In [2]:
# Example usage:
accession_numbers = [
    "SRR23891364", "SRR23891365", "SRR23891368",
    "SRR23891369", "SRR23891371", "SRR23891373",
    "SRR23891374", "SRR23891375", "SRR23891376",
    "SRR23891377", "SRR23891378", "SRR23891363",
    "SRR23891366", "SRR23891367", "SRR23891370",
    "SRR23891372"
]


In [ ]:

def download_data(accession_numbers, output_directory):
    for accession_number in accession_numbers:
        command = f"fastq-dump -O {output_directory} {accession_number}"
        os.system(command)

output_directory = "SRA_data/"
download_data(accession_numbers, output_directory)


In [ ]:
for accession_number in accession_numbers:
    !gzip SRA_data/{accession_number}.fastq

# Trimming

In [ ]:

for accession_number in accession_numbers:
    # Create the output directory if it doesn't exist
    trimmed_output_dir='no_length/processed/trimmed/'
    os.makedirs(trimmed_output_dir, exist_ok=True)
    input_path = os.path.join("SRA_data", f"{accession_number}.fastq.gz")
    output_path = os.path.join(trimmed_output_dir, accession_number)
    !trim_galore --quality 5 --output_dir $output_path $input_path

# Mapping

Creating Index for non-mrna 

In [6]:
os.makedirs("no_length/non-mrna/non-mRNA_ref/", exist_ok=True)
!STAR --runMode genomeGenerate \
--genomeDir no_length/non-mrna/non-mRNA_ref/ \
--genomeFastaFiles no_length/non-mrna/non-mRNAfasta/C_albicans_SC5314_A22_current_other_features_no_introns.fasta \
--runThreadN 16

	/usr/lib/rna-star/bin/STAR-avx2 --runMode genomeGenerate --genomeDir no_length/non-mrna/non-mRNA_ref/ --genomeFastaFiles no_length/non-mrna/non-mRNAfasta/C_albicans_SC5314_A22_current_other_features_no_introns.fasta --runThreadN 16
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 12:08:01 ..... started STAR run
Dec 06 12:08:01 ... starting to generate Genome files
!!!!! WARNING: --genomeSAindexNbases 14 is too large for the genome size=806909, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 8
Dec 06 12:08:01 ... starting to sort Suffix Array. This may take a long time...
Dec 06 12:08:01 ... sorting Suffix Array chunks and saving them to disk...
Dec 06 12:08:04 ... loading chunks from disk, packing SA...
Dec 06 12:08:04 ... finished generating suffix array
Dec 06 12:08:04 ... generating Suffix Array index
Dec 06 12:08:06 ... completed Suffix Array index
Dec 06 12:08:06 ..

There are some annotation problems in the current GTF file, hence it is always recommended to check whether your gtf file is properly formatted or not. 

In [ ]:
!sed 's/""/"/g' Gtf/DatasetS3.gtf > Gtf/DatasetS3_1.gtf
!sed 's/"transcript_id/transcript_id/; s/;"/;/' Gtf/DatasetS3_1.gtf> Gtf/DatasetS3_final.gtf

Creating index for mrna 

In [7]:
os.makedirs('no_length/mrna/ref', exist_ok=True)
# build genome index using GTF file
!STAR --runThreadN 16 \
     --runMode genomeGenerate \
     --genomeDir no_length/mrna/ref/ \
     --genomeSAindexNbases 11 \
     --sjdbGTFfeatureExon exon\
     --genomeFastaFiles no_length/Assembly_22_fasta/C_albicans_SC5314_A22_current_chromosomes.fasta \
     --sjdbGTFfile no_length/Gtf/DatasetS3_final.gtf

	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --runMode genomeGenerate --genomeDir no_length/mrna/ref/ --genomeSAindexNbases 11 --sjdbGTFfeatureExon exon --genomeFastaFiles no_length/Assembly_22_fasta/C_albicans_SC5314_A22_current_chromosomes.fasta --sjdbGTFfile no_length/Gtf/DatasetS3_final.gtf
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 12:12:05 ..... started STAR run
Dec 06 12:12:05 ... starting to generate Genome files
Dec 06 12:12:05 ..... processing annotations GTF
Dec 06 12:12:05 ... starting to sort Suffix Array. This may take a long time...
Dec 06 12:12:06 ... sorting Suffix Array chunks and saving them to disk...
Dec 06 12:12:50 ... loading chunks from disk, packing SA...
Dec 06 12:12:50 ... finished generating suffix array
Dec 06 12:12:50 ... generating Suffix Array index
Dec 06 12:12:52 ... completed Suffix Array index
Dec 06 12:12:52 ..... inserting junctions into the genome indices
Dec 06 12:12:52 ... writing Ge

non-mRNA mapping

In [23]:

trimmed_fastq_directory=f'no_length/processed/trimmed/'
# Define the paths to your trimmed FASTQ files
suffix= '_trimmed.fq.gz'
trimmed_fastq_files = [os.path.join(trimmed_fastq_directory,str(accession_number), str(accession_number)+suffix) for accession_number in accession_numbers]
print(trimmed_fastq_files)

['no_length/processed/trimmed/SRR23891364/SRR23891364_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891365/SRR23891365_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891368/SRR23891368_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891369/SRR23891369_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891373/SRR23891373_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891375/SRR23891375_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891376/SRR23891376_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891377/SRR23891377_trimmed.fq.gz', 'no_length/processed/trimmed/SRR23891378/SRR23891378_trimmed.fq.gz']


In [4]:
import os

accession_numbers = [
    "SRR23891367", "SRR23891370", "SRR23891372" 
] 

trimmed_fastq_directory=f'no_length/processed/trimmed/'

# Define the paths to your trimmed FASTQ files
suffix= '_trimmed.fq.gz'
trimmed_fastq_files = [os.path.join(trimmed_fastq_directory,str(accession_number), str(accession_number)+suffix) for accession_number in accession_numbers]


# Define the output directories
output_bam_dir = 'no_length/non-mrna/mapped_reads/bams/'
output_unmapped_fastq_dir = 'no_length/non-mrna/mapped_reads/unmapped_fastq/'
output_starlogs_dir = 'no_length/non-mrna/mapped_reads/starlogs/'
# temp_dir = '/tmp/mergedrRNA/'

# Create the temporary directory
# os.makedirs(temp_dir, exist_ok=True)
# Create the output directories if they don't exist
os.makedirs(output_bam_dir, exist_ok=True)
os.makedirs(output_unmapped_fastq_dir, exist_ok=True)
os.makedirs(output_starlogs_dir, exist_ok=True)

# Rule: map_star_rRNA
for trimmed_fastq_file in trimmed_fastq_files:
    sample_name = os.path.basename(trimmed_fastq_file).split('_trimmed.fq.gz')[0]
    output_bam = os.path.join(output_bam_dir, f'{sample_name}.bam')
    output_unmapped_fastq = os.path.join(output_unmapped_fastq_dir, f'{sample_name}.unmapped_rRNA.fastq.gz')
    output_starlogs = os.path.join(output_starlogs_dir, f'{sample_name}_starlogs')

    # STAR command for read mapping
    star_cmd = (
        f'STAR --runThreadN 16 '
        f'--genomeDir no_length/non-mrna/non-mRNA_ref '
        f'--outFilterMismatchNmax 2 '
        f'--outFileNamePrefix {output_bam.replace(".bam", ".map_rRNA_")} '
        f'--readFilesIn {trimmed_fastq_file} '
        f'--outReadsUnmapped Fastx '
        f'--readFilesCommand zcat ' 
        f'--outSAMtype BAM SortedByCoordinate '
        f'--limitBAMsortRAM 9659450608'
    )

    # Run the STAR command
    os.system(star_cmd)

    print(f"Mapping completed for {sample_name}. Output BAM: {output_bam}")

# Print a message indicating the completion
print("Read mapping process completed.")

	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/non-mrna/non-mRNA_ref --outFilterMismatchNmax 2 --outFileNamePrefix no_length/non-mrna/mapped_reads/bams/SRR23891375.map_rRNA_ --readFilesIn no_length/processed/trimmed/SRR23891375/SRR23891375_trimmed.fq.gz --outReadsUnmapped Fastx --readFilesCommand zcat --limitBAMsortRAM 9659450608--outSAMtype BAM SortedByCoordinate
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 15:58:11 ..... started STAR run
Dec 06 15:58:11 ..... loading genome
Dec 06 15:58:13 ..... started mapping
Dec 06 16:17:21 ..... finished mapping
Dec 06 16:17:21 ..... finished successfully
Mapping completed for SRR23891375. Output BAM: no_length/non-mrna/mapped_reads/bams/SRR23891375.bam
	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/non-mrna/non-mRNA_ref --outFilterMismatchNmax 2 --outFileNamePrefix no_length/non-mrna/mapped_reads/bams/SRR23891376.map_rRNA_ --readFilesIn no_le

In [6]:
# accession_numbers = [
#     "SRR23891375", "SRR23891376" 
# ] 

# for accession in accession_numbers:
#     !samtools sort no_length/non-mrna/mapped_reads/bams/{accession}.map_rRNA_Aligned.out.sam -o no_length/non-mrna/mapped_reads/bams/{accession}.map_rRNA_Aligned.sortedByCoord.out.bam
    

[bam_sort_core] merging from 46 files and 1 in-memory blocks...
[E::aux_parse] Incomplete aux field
[W::sam_read1_sam] Parse error at line 32205770
samtools sort: truncated file. Aborting


In [12]:
for accession_number in accession_numbers:
    !mv 'no_length/non-mrna/mapped_reads/bams/{accession_number}.map_rRNA_Unmapped.out.mate1' "no_length/non-mrna/mapped_reads/unmapped_fastq/{accession_number}.unmapped_rRNA.fastq"

mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891364.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891365.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891368.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891369.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891373.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891375.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891377.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_length/non-mrna/mapped_reads/bams/SRR23891378.map_rRNA_Unmapped.out.mate1': No such file or directory
mv: cannot stat 'no_leng

In [13]:
for accession_number in accession_numbers:
    !gzip "no_length/non-mrna/mapped_reads/unmapped_fastq/{accession_number}.unmapped_rRNA.fastq"
    !gzip "no_length/non-mrna/mapped_reads/bams/{accession_number}.map_rRNA_Aligned.sortedByCoord.out.bam"

^C


In [12]:
import os
# Define the paths to your trimmed FASTQ files
unmapped_fastq_directory = 'no_length/non-mrna/mapped_reads/unmapped_fastq/'
suffix= '.unmapped_rRNA.fastq.gz'
# Get a list of all unmapped FastQ.gz files in the directory
fastq_files = [os.path.join(unmapped_fastq_directory,str(accession_number)+suffix) for accession_number in accession_numbers]
print(fastq_files)

['no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891364.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891365.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891368.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891369.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891373.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891375.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891377.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891378.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891366.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891370.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891372.unmapped_rRNA.fastq.gz', 'no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891367.unma

no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891367_unmapped_rRNA.fastq.gz
no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891367.unmapped_rRNA.fastq.gz

mRNA mapping

In [13]:
import os
# Define the paths to your input FASTQ files
unmapped_fastq_directory = 'no_length/non-mrna/mapped_reads/unmapped_fastq/'
suffix= '.unmapped_rRNA.fastq.gz'
# Get a list of all unmapped FastQ.gz files in the directory
fastq_files = [os.path.join(unmapped_fastq_directory,str(accession_number)+suffix) for accession_number in accession_numbers]


# Define the STAR index path
star_index = 'no_length/mrna/ref'

# Define the output directories
output_dir_mapped = 'no_length/mapped_mRNA/bams_star/'
# output_dir_unmapped = 'no_length/unmapped_mRNA/fastq/'
output_dir_logs = 'no_length/mapped_mRNA/starlogs/'

# Create the output directories if they don't exist
os.makedirs(output_dir_mapped, exist_ok=True)
# os.makedirs(output_dir_unmapped, exist_ok=True)
os.makedirs(output_dir_logs, exist_ok=True)

# Rule: map_star_mRNA
# Map the unmapped RNA reads to the mRNA index using STAR
for fastq_file in fastq_files:
    sample_name = os.path.basename(fastq_file).split('.')[0]
    output_bam = os.path.join(output_dir_mapped, f'{sample_name}.bam')
    # output_fastq = os.path.join(output_dir_unmapped, f'{sample_name}.unmapped_mRNA.fastq.gz')
    output_logs = os.path.join(output_dir_logs, f'{sample_name}.map_mRNALog')

    star_cmd = f'''
        STAR --runThreadN 16 \
        --genomeDir {star_index} \
        --outFilterMismatchNmax 2 \
        --readFilesCommand zcat \
        --outFileNamePrefix {output_bam.replace(".bam", ".map_mRNA")} --readFilesIn {fastq_file} \
        --alignEndsType EndToEnd \
        --outSAMtype BAM SortedByCoordinate \
        --limitBAMsortRAM 18659450608 \
    '''
    
    os.system(star_cmd)

#     # Print the paths of the output files
#     print("Mapped BAM file:", output_bam)
#     # print("Unmapped FASTQ file:", output_fastq)

	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/mrna/ref --outFilterMismatchNmax 2 --readFilesCommand zcat --outFileNamePrefix no_length/mapped_mRNA/bams_star/SRR23891364.map_mRNA --readFilesIn no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891364.unmapped_rRNA.fastq.gz --alignEndsType EndToEnd --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 18659450608
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 19:50:21 ..... started STAR run
Dec 06 19:50:21 ..... loading genome
Dec 06 19:50:21 ..... started mapping
Dec 06 19:52:40 ..... finished mapping
Dec 06 19:52:40 ..... started sorting BAM
Dec 06 19:53:53 ..... finished successfully


[E::hts_open_format] Failed to open file "no_length/mapped_mRNA/bams_star/SRR23891364.bam" : No such file or directory
samtools index: failed to open "no_length/mapped_mRNA/bams_star/SRR23891364.bam": No such file or directory


Mapped BAM file: no_length/mapped_mRNA/bams_star/SRR23891364.bam
	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/mrna/ref --outFilterMismatchNmax 2 --readFilesCommand zcat --outFileNamePrefix no_length/mapped_mRNA/bams_star/SRR23891365.map_mRNA --readFilesIn no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891365.unmapped_rRNA.fastq.gz --alignEndsType EndToEnd --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 18659450608
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 19:53:53 ..... started STAR run
Dec 06 19:53:53 ..... loading genome
Dec 06 19:53:54 ..... started mapping
Dec 06 19:56:10 ..... finished mapping
Dec 06 19:56:10 ..... started sorting BAM
Dec 06 19:56:54 ..... finished successfully


[E::hts_open_format] Failed to open file "no_length/mapped_mRNA/bams_star/SRR23891365.bam" : No such file or directory
samtools index: failed to open "no_length/mapped_mRNA/bams_star/SRR23891365.bam": No such file or directory


Mapped BAM file: no_length/mapped_mRNA/bams_star/SRR23891365.bam
	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/mrna/ref --outFilterMismatchNmax 2 --readFilesCommand zcat --outFileNamePrefix no_length/mapped_mRNA/bams_star/SRR23891368.map_mRNA --readFilesIn no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891368.unmapped_rRNA.fastq.gz --alignEndsType EndToEnd --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 18659450608
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 19:56:55 ..... started STAR run
Dec 06 19:56:55 ..... loading genome
Dec 06 19:56:55 ..... started mapping
Dec 06 21:02:36 ..... finished mapping
Dec 06 21:02:37 ..... started sorting BAM
Dec 06 21:03:00 ..... finished successfully


...

[E::hts_open_format] Failed to open file "no_length/mapped_mRNA/bams_star/SRR23891370.bam" : No such file or directory
samtools index: failed to open "no_length/mapped_mRNA/bams_star/SRR23891370.bam": No such file or directory


Mapped BAM file: no_length/mapped_mRNA/bams_star/SRR23891370.bam
	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/mrna/ref --outFilterMismatchNmax 2 --readFilesCommand zcat --outFileNamePrefix no_length/mapped_mRNA/bams_star/SRR23891372.map_mRNA --readFilesIn no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891372.unmapped_rRNA.fastq.gz --alignEndsType EndToEnd --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 18659450608
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 22:37:46 ..... started STAR run
Dec 06 22:37:46 ..... loading genome
Dec 06 22:37:47 ..... started mapping
Dec 06 22:39:57 ..... finished mapping
Dec 06 22:39:57 ..... started sorting BAM
Dec 06 22:40:23 ..... finished successfully


[E::hts_open_format] Failed to open file "no_length/mapped_mRNA/bams_star/SRR23891372.bam" : No such file or directory
samtools index: failed to open "no_length/mapped_mRNA/bams_star/SRR23891372.bam": No such file or directory


Mapped BAM file: no_length/mapped_mRNA/bams_star/SRR23891372.bam
	/usr/lib/rna-star/bin/STAR-avx2 --runThreadN 16 --genomeDir no_length/mrna/ref --outFilterMismatchNmax 2 --readFilesCommand zcat --outFileNamePrefix no_length/mapped_mRNA/bams_star/SRR23891367.map_mRNA --readFilesIn no_length/non-mrna/mapped_reads/unmapped_fastq/SRR23891367.unmapped_rRNA.fastq.gz --alignEndsType EndToEnd --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 18659450608
	STAR version: 2.7.10a   compiled: 2022-01-16T16:35:44+00:00 <place not set in Debian package>
Dec 06 22:40:23 ..... started STAR run
Dec 06 22:40:23 ..... loading genome
Dec 06 22:40:23 ..... started mapping
Dec 06 23:03:29 ..... finished mapping
Dec 06 23:03:29 ..... started sorting BAM
Dec 06 23:03:42 ..... finished successfully
Mapped BAM file: no_length/mapped_mRNA/bams_star/SRR23891367.bam


[E::hts_open_format] Failed to open file "no_length/mapped_mRNA/bams_star/SRR23891367.bam" : No such file or directory
samtools index: failed to open "no_length/mapped_mRNA/bams_star/SRR23891367.bam": No such file or directory


In [ ]:
# for accession_number in accession_numbers:
#     !mv 'no_length/mapped_mRNA/bams_star/{accession_number}_unmapped_rRNA.map_mRNAUnmapped.out.mate1' "no_length/unmapped_mRNA/fastq/{accession_number}_unmapped_rRNA.fastq"

In [ ]:
# for accession_number in accession_numbers:
    # !gzip "no_length/unmapped_mRNA/fastq/{accession_number}_unmapped_rRNA.fastq"

# Filter Unique Mappings

In [4]:
import subprocess
import os

# Set the paths
input_dir = 'no_length/mapped_mRNA/bams_star/'
output_dir = 'no_length/mapped_mRNA/bams_NH1/'
tmp_dir = '/tmp'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)


# Create a sample_list          
sample_list = [f'{number}.map_mRNAAligned.sortedByCoord.out' for number in accession_numbers]

# Iterate over samples
for sample in sample_list:
    input_bam = os.path.join(input_dir, f'{sample}.bam')
    output_bam = os.path.join(output_dir, f'{sample}.NH1.bam')
    output_unsorted_bam = f'{output_bam}.unsorted'

    # Rule: filter_uniq_mapping
    cmd = f"""
        bamtools filter -tag NH:1 -mapQuality ">=255" -mapQuality "<256" -in {input_bam} -out {output_bam}.unsorted &&
        samtools sort -T {tmp_dir}/{sample}_uniq_mapping -o {output_bam} {output_bam}.unsorted &&
        samtools index {output_bam} &&
        rm {output_bam}.unsorted
    """

    try:
        # Run the command
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while processing {sample}: {e}")


# Filter Twice Mappings

In [6]:
import subprocess
import os

# Set the paths
input_dir = 'no_length/mapped_mRNA/bams_star/'
output_dir = 'no_length/mapped_mRNA/bams_NH2/'
tmp_dir = '/tmp'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)


# List of samples
sample_list = [f'{number}.map_mRNAAligned.sortedByCoord.out' for number in accession_numbers] 

# Iterate over samples
for sample in sample_list:
    input_bam = os.path.join(input_dir, f'{sample}.bam')
    output_bam = os.path.join(output_dir, f'{sample}.NH2.bam')
    output_unsorted_bam = f'{output_bam}.unsorted'

    # Rule: filter_twice_mapping
    # Based on https://groups.google.com/d/msg/rna-star/FhVk4f61Vcg/osx6DLTpAgAJ
    cmd = f"""
        bamtools filter -tag NH:2 -mapQuality "<255" -in {input_bam} -out {output_unsorted_bam} &&
        samtools sort -T {tmp_dir}/{sample}_twice_mapping -o {output_bam} {output_unsorted_bam} &&
        samtools index {output_bam} &&
        rm {output_unsorted_bam}
    """

    try:
        # Run the command
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while processing {sample}: {e}")

[bam_sort_core] merging from 8 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 6 files and 1 in-memory blocks...
[bam_sort_core] merging from 5 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 5 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 8 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...


# Collapsing NH2 to NH1

In [11]:
# pip install pysam
import pysam  
import sys

input_dir = 'no_length/mapped_mRNA/bams_NH2/'
output_dir = 'no_length/mapped_mRNA/bams_NH2ToNH1/'
threads = 16  # Replace with the actual number of threads
tmp_dir = '/tmp'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)

def collapse_nh2_to_nh1(input_bam, output_bam):
    # Open the input BAM file
    with pysam.AlignmentFile(input_bam, "rb") as in_bam:
        # Open the output BAM file
        with pysam.AlignmentFile(output_bam, "wb", header=in_bam.header) as out_bam:
            # Keep track of read names to only output one alignment per read
            seen_reads = set()

            # Iterate through each read in the input BAM file
            for read in in_bam:
                # Check if the read has the NH tag equal to 2
                if read.get_tag("NH") == 2:
                    read_name = read.query_name

                    # If this read has not been seen before, output it
                    if read_name not in seen_reads:
                        # Change the NH tag to 1
                        read.set_tag("NH", 1)
                        out_bam.write(read)
                        seen_reads.add(read_name)
    return


# List of samples
sample_list = [f'{number}.map_mRNAAligned.sortedByCoord.out' for number in accession_numbers] # Replace with your actual sample names

for sample in sample_list:

    input_bam = os.path.join(input_dir, f'{sample}.NH2.bam')
    output_bam = os.path.join(output_dir, f'{sample}.NH2ToNH1.bam.unsorted')
    collapse_nh2_to_nh1(input_bam, output_bam)

Sorting NH2ToNH1

In [12]:
import subprocess
import os

# Set the paths
input_dir = 'no_length/mapped_mRNA/bams_NH2/'
output_dir = 'no_length/mapped_mRNA/bams_NH2ToNH1/'
threads = 16  
tmp_dir = '/tmp'

os.makedirs(output_dir, exist_ok=True)                 
os.makedirs(tmp_dir, exist_ok=True)

# List all BAM files in the input directory
sample_list = [f'{number}.map_mRNAAligned.sortedByCoord.out' for number in accession_numbers] # Replace with your actual sample names


for sample in sample_list:

    input_bam = os.path.join(input_dir, f'{sample}.NH2.bam')
    output_bam = os.path.join(output_dir, f'{sample}.NH2ToNH1.bam')
    output_unsorted_bam = f'{output_bam}.unsorted'

    # Rule: filter_twice_mapping
    # Based on https://groups.google.com/d/msg/rna-star/FhVk4f61Vcg/osx6DLTpAgAJ
    cmd = f"""
        samtools index {input_bam} &&
        samtools sort -T {tmp_dir}/{sample}_collapse -o {output_bam} {output_bam}.unsorted &&
        samtools index {output_bam} &&
        rm {output_bam}.unsorted
    """

    try:
        # Run the command
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while processing {sample_name}: {e}")


[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...


Merging NH2ToNH1 with NH1

In [13]:
import subprocess
import os

# Set the paths
input_dir_nh2nh1 = 'no_length/mapped_mRNA/bams_NH2ToNH1/'
input_dir_nh1 = 'no_length/mapped_mRNA/bams_NH1/'
output_dir_collapsed = 'no_length/mapped_mRNA/bams_collapsed/'
threads = 16  # Replace with the actual number of threads
tmp_dir = '/tmp'

os.makedirs(output_dir_collapsed, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)                                        

# List all NH2ToNH1 BAM files in the input directory
sample_list = [f'{number}.map_mRNAAligned.sortedByCoord.out' for number in accession_numbers] # Replace with your actual sample names

# Iterate through each NH2ToNH1 BAM file
for sample in sample_list:
    
    # Construct the full paths for NH2ToNH1 and NH1 BAM files
    input_bam_nh2nh1 = os.path.join(input_dir_nh2nh1, f'{sample}.NH2ToNH1.bam')
    input_bam_nh1 = os.path.join(input_dir_nh1, f'{sample}.NH1.bam')
    output_bam_collapsed = os.path.join(output_dir_collapsed, f'{sample}.bam')
   

    # Rule: merge_NH2NH1_NH1
    cmd = f"""
        bamtools merge -in {input_bam_nh2nh1} -in {input_bam_nh1} -out {output_bam_collapsed}.unsorted &&
        samtools sort -T {tmp_dir}/{sample}_collapsed -o {output_bam_collapsed} {output_bam_collapsed}.unsorted &&
        samtools index {output_bam_collapsed} &&
        rm {output_bam_collapsed}.unsorted
    """

    try:
        # Run the command
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while processing {sample_name}: {e}")

[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 1 files and 1 in-memory blocks...


# FeatureCount


Separating GTF into hapA and hapB

In [27]:
!grep 'gene_id ".*_A"' no_length/Gtf/DatasetS3_final.gtf > no_length/Gtf/GTF_hapA.gtf
!grep 'gene_id ".*_B"' no_length/Gtf/DatasetS3_final.gtf > no_length/Gtf/GTF_hapB.gtf

Now, we also have gene-ids which do not have hapA and hapB! Hence, now I'll be removing those. 
Also, the gene-id "MSTRG" is not needed for further processes.

In [28]:
!grep -v 'gene_id "MSTRG' no_length/Gtf/DatasetS3_final.gtf > no_length/Gtf/GTF_noM.gtf
!grep -v 'gene_id ".*_[AB]"' no_length/Gtf/GTF_noM.gtf > no_length/Gtf/GTF_noAB.gtf

Using featureCounts with hapA gtf :-

In [24]:
# GTF file
gtf_file = "no_length/Gtf/GTF_hapA.gtf"

# Output directory for featureCounts results                      
output_counts_dir = "no_length/reads_gene/hapA"

# Create the output directory if it doesn't exist
!mkdir -p $output_counts_dir

# Iterate through accession numbers
for accession_number in accession_numbers:                        
    # Input BAM file
    input_bam = f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"
    
    # Output counts file
    output_counts = f"{output_counts_dir}/counts_hapA_{accession_number}.txt"
    # Run featureCounts command
    !featureCounts -g gene_id -a $gtf_file -o $output_counts $input_bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           SRR23891364.map_mRNAAligned.sortedByCoord.ou ... ||
||                                                                            ||
||             Output file : counts_hapA_SRR23891364.txt         

Now working with the hapB gtf file:-

In [25]:
# GTF file
gtf_file = "no_length/Gtf/GTF_hapB.gtf"

# Output directory for featureCounts results                      
output_counts_dir = "no_length/reads_gene/hapB"

# Create the output directory if it doesn't exist
!mkdir -p $output_counts_dir

# Iterate through accession numbers
for accession_number in accession_numbers:                        
    # Input BAM file
    input_bam = f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"
    
    # Output counts file
    output_counts = f"{output_counts_dir}/counts_hapB_{accession_number}.txt"
    # Run featureCounts command
    !featureCounts -g gene_id -a $gtf_file -o $output_counts $input_bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           SRR23891364.map_mRNAAligned.sortedByCoord.ou ... ||
||                                                                            ||
||             Output file : counts_hapB_SRR23891364.txt         

Now working with the no_hapAB gtf file:-

In [26]:
# GTF file
gtf_file = "no_length/Gtf/GTF_noAB.gtf"

# Output directory for featureCounts results                      
output_counts_dir = "no_length/reads_gene/nohapAB"

# Create the output directory if it doesn't exist
!mkdir -p $output_counts_dir

# Iterate through accession numbers
for accession_number in accession_numbers:                        
    # Input BAM file
    input_bam = f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"
    
    # Output counts file
    output_counts = f"{output_counts_dir}/counts_nohapAB_{accession_number}.txt"
    # Run featureCounts command
    !featureCounts -g gene_id -a $gtf_file -o $output_counts $input_bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           SRR23891364.map_mRNAAligned.sortedByCoord.ou ... ||
||                                                                            ||
||             Output file : counts_nohapAB_SRR23891364.txt      

Converting txt to csv files for ease of processing

In [7]:
import pandas as pd
import os
for accession_number in accession_numbers:

    out_dir1= "no_length/reads_gene/hapA/csv"
    os.makedirs(out_dir1, exist_ok=True)
    out_dir2= "no_length/reads_gene/hapB/csv"
    os.makedirs(out_dir2, exist_ok=True)
    out_dir3= "no_length/reads_gene/nohapAB/csv"
    os.makedirs(out_dir3, exist_ok=True)

    df = pd.read_csv(f"no_length/reads_gene/hapA/counts_hapA_{accession_number}.txt", skiprows=1)
    df.to_csv(f'{out_dir1}/counts_hapA_{accession_number}.csv', index=False)

    df1 = pd.read_csv(f"no_length/reads_gene/hapB/counts_hapB_{accession_number}.txt", skiprows=1)
    df1.to_csv(f'{out_dir2}/counts_hapB_{accession_number}.csv', index=False)

    df2 = pd.read_csv(f'no_length/reads_gene/nohapAB/counts_nohapAB_{accession_number}.txt', skiprows=1)
    df2.to_csv(f'{out_dir3}/counts_nohapAB_{accession_number}.csv', index=False)


Now we want to add the counts for hapA and hapB gene-ids and then concatinate the resulting counts with the nohapAB counts.

In [14]:
import pandas as pd

dfs=[]


for accession_number in accession_numbers:      
    # Read the files for hapA and hapB 
    df1 = pd.read_csv(f"no_length/reads_gene/hapA/csv/counts_hapA_{accession_number}.csv", sep="\t") 
    df2 = pd.read_csv(f"no_length/reads_gene/hapB/csv/counts_hapB_{accession_number}.csv", sep="\t")


    # Remove the last two characters from the Geneid column              

    df1["Geneid"] = df1["Geneid"].str.slice(0, -2)
    df2["Geneid"] = df2["Geneid"].str.slice(0, -2)
        
    # df_len= df1[["Geneid","Length"]]  ## exracting length for further DeSEQ and TE analysis

    # Extract the columns Geneid and reads
    df1 = df1[["Geneid", f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"]]
    df2 = df2[["Geneid", f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"]]

  

    # Rename the reads column
    df1 = df1.rename(columns={f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam": f"reads_{accession_number}_x"})
    df2 = df2.rename(columns={f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam": f"reads_{accession_number}_y"})

    # Merge the two dataframes on Geneid
    df = pd.merge(df1, df2, on="Geneid", how="inner")

    # Convert the "reads" columns to numeric
    df[f"reads_{accession_number}_x"] = pd.to_numeric(df[f"reads_{accession_number}_x"], errors='coerce')
    df[f"reads_{accession_number}_y"] = pd.to_numeric(df[f"reads_{accession_number}_y"], errors='coerce')

    # Sum the reads columns
    df[f"reads_{accession_number}"] = df[f"reads_{accession_number}_x"] + df[f"reads_{accession_number}_y"]

    # Drop the redundant reads columns
    df = df.drop(columns=[f"reads_{accession_number}_x", f"reads_{accession_number}_y"])

    # Append the dataframe to the list
    dfs.append(df)

df_final = pd.concat(dfs, axis=1)

# Drop the duplicate Geneid columns
df_final = df_final.loc[:, ~df_final.columns.duplicated()]
# df_len.to_csv("no_length/reads_gene/length_AB.csv", sep="\t", index=False)
# Write the dataframe to a new file
df_final.to_csv("no_length/reads_gene/counts_sum.csv", sep="\t", index=False)

Now we want to concatinate the above csv file with the nohapAB csv files

In [15]:
dfss=[]

for accession_number in accession_numbers:
    df= pd.read_csv(f"no_length/reads_gene/nohapAB/csv/counts_nohapAB_{accession_number}.csv", sep="\t")

    # df_len=df[["Geneid", "Length"]]

    df = df[["Geneid", f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"]]
    

    # Rename the reads column
    df = df.rename(columns={f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam": f'reads_{accession_number}'})

    dfss.append(df)

df_final = pd.concat(dfss, axis=1)
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# Write the dataframe to a new file
df_final.to_csv("no_length/reads_gene/counts_sum_nohapAB.csv", sep="\t", index=False)
# df_len.to_csv("no_length/reads_gene/length_noAB.csv", sep="\t", index=False)


In [42]:
import pandas as pd

# Read the first CSV file
file1_path = 'no_length/reads_gene/counts_sum_nohapAB.csv'
df1 = pd.read_csv(file1_path, sep='\t')

# Read the second CSV file
file2_path = 'no_length/reads_gene/counts_sum.csv'
df2 = pd.read_csv(file2_path, sep='\t')

# Concatenate the dataframes vertically
result_df = pd.concat([df2, df1], ignore_index=True, sort=False)

result_df.to_csv('no_length/reads_gene/counts_sum1.csv', sep='\t', index=False)

# Display the result or save it to a new CSV file
print(result_df)

         Geneid  reads_SRR23891364  reads_SRR23891365  reads_SRR23891368  \
0     C1_00010W                  0                  0                 81   
1     C1_00020C                  0                  0                 12   
2     C1_00030C                  0                  0                  0   
3     C1_00040W                 67                113                 40   
4     C1_00050C                  0                  0                  5   
...         ...                ...                ...                ...   
6194  CM_00340W              11561              13861                222   
6195  CM_00370W               5110               4366                494   
6196  CM_00380W              36826              35523               2923   
6197  CM_00440W              20734              22667               2157   
6198  CM_00520W                  8                 33                  1   

      reads_SRR23891369  reads_SRR23891371  reads_SRR23891373  \
0                     

Now there can also be some geneids existing just for hapA or hapB. We want these gene_ids as well. :-


In [16]:
import pandas as pd


dfs_hapA = []
dfs_hapB = []                        ## By doing this we are making a two different csv files which will have hapA and hapB geneids respectively.

for accession_number in accession_numbers:
    df1=pd.read_csv(f'no_length/reads_gene/hapA/csv/counts_hapA_{accession_number}.csv',sep="\t")
    df2=pd.read_csv(f'no_length/reads_gene/hapB/csv/counts_hapB_{accession_number}.csv',sep="\t")

    df1["Geneid"] = df1["Geneid"].str.slice(0, -2)
    df2["Geneid"] = df2["Geneid"].str.slice(0, -2)

    df_lenA=df1[["Geneid", "Length"]]
    df_lenB=df2[["Geneid", "Length"]]
    # Extract the columns Geneid and reads
    df1 = df1[["Geneid", f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"]]
    df2 = df2[["Geneid", f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam"]]

    # Rename the reads column
    df1 = df1.rename(columns={f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam": f"reads_{accession_number}"})
    df2 = df2.rename(columns={f"no_length/mapped_mRNA/bams_collapsed/{accession_number}.map_mRNAAligned.sortedByCoord.out.bam": f"reads_{accession_number}"})

    dfs_hapA.append(df1)
    dfs_hapB.append(df2)

# Concatenate DataFrames for hapA and hapB
df_hapA = pd.concat(dfs_hapA, axis=1)
df_hapB = pd.concat(dfs_hapB, axis=1)

df_hapA = df_hapA.loc[:, ~df_hapA.columns.duplicated()]           ## removing duplicated columns
df_hapB = df_hapB.loc[:, ~df_hapB.columns.duplicated()]

df_hapA.to_csv("no_length/reads_gene/counts_sumA.csv", sep="\t", index=False)
df_hapB.to_csv("no_length/reads_gene/counts_sumB.csv", sep="\t", index=False)

df_lenA.to_csv("no_length/reads_gene/length_A.csv", sep="\t", index=False)
df_lenB.to_csv("no_length/reads_gene/length_B.csv", sep="\t", index=False)

In [22]:
df_hapA

,Geneid,reads_SRR23891364,reads_SRR23891365,reads_SRR23891368,reads_SRR23891369,reads_SRR23891371,reads_SRR23891373,reads_SRR23891374,reads_SRR23891375,reads_SRR23891376,reads_SRR23891377,reads_SRR23891378,reads_SRR23891363,reads_SRR23891366,reads_SRR23891367,reads_SRR23891370,reads_SRR23891372
0,C1_00010W,0,0,72,2,46,0,14,48,41,7,8,0,0,42,3,10
1,C1_00020C,0,0,12,0,0,0,22,0,1,2,1,0,0,31,0,29
2,C1_00030C,0,0,0,0,0,0,0,0,1,3,1,0,0,1,0,0
3,C1_00040W,67,113,40,41,67,75,25,5,5,42,3,15,0,1,18,54
4,C1_00050C,0,0,5,0,1,3,0,4,3,0,0,60,260,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6180,CR_10820W,631,236,258,26,249,306,129,103,50,41,57,356,172,131,46,306
6181,CR_10830C,753,445,872,1484,651,258,479,1756,985,907,292,548,0,537,306,592
6182,CR_10840C,19494,10371,24680,6560,21799,16328,24314,19324,25536,13230,14085,10882,25041,5675,9923,19744
6183,CR_10850C,7766,6711,10454,9884,10842,11125,11011,8051,9387,7149,5004,7287,6349,4522,2708,11150


from the above csv files we will extract the geneids of hapB which are not present in hapA and vice versa.

In [29]:
df_hapA=pd.read_csv("no_length/reads_gene/counts_sumA.csv", sep='\t')
df_hapB=pd.read_csv("no_length/reads_gene/counts_sumB.csv", sep='\t')

diff1= df_hapA[~df_hapA["Geneid"].isin(df_hapB["Geneid"])]
diff2= df_hapB[~df_hapB["Geneid"].isin(df_hapA["Geneid"])]

diff1 = diff1.loc[:, ~diff1.columns.duplicated()]
diff2 = diff2.loc[:, ~diff2.columns.duplicated()]

diff1.to_csv("no_length/reads_gene/diff1.csv", sep="\t", index=False)
diff2.to_csv("no_length/reads_gene/diff2.csv", sep="\t", index=False)

In [24]:
diff1      ## geneids of hapA which are not there in hapB

,Geneid,reads_SRR23891364,reads_SRR23891365,reads_SRR23891368,reads_SRR23891369,reads_SRR23891371,reads_SRR23891373,reads_SRR23891374,reads_SRR23891375,reads_SRR23891376,reads_SRR23891377,reads_SRR23891378,reads_SRR23891363,reads_SRR23891366,reads_SRR23891367,reads_SRR23891370,reads_SRR23891372
3986,C5_01740C,379,165,227,649,411,501,527,591,331,391,194,207,8,142,35,555
3987,C5_01750C,0,22,0,1,0,25,26,3,0,0,9,6,1,6,1,3
3988,C5_01760C,1086,498,524,761,1480,1588,1361,1558,1459,691,580,787,2784,520,181,1425
3989,C5_01770C,3746,3037,1934,1782,3566,4422,3889,1487,2260,1691,1722,2580,3892,542,688,4158
3990,C5_01780W,1354,830,467,80,1097,1152,1087,758,232,163,320,1043,2926,267,148,1304
3991,C5_01790C,139,15,134,142,35,43,129,179,58,26,46,19,1,190,25,105


In [25]:
diff2       ## geneids of hapB which are not there in hapA

,Geneid,reads_SRR23891364,reads_SRR23891365,reads_SRR23891368,reads_SRR23891369,reads_SRR23891371,reads_SRR23891373,reads_SRR23891374,reads_SRR23891375,reads_SRR23891376,reads_SRR23891377,reads_SRR23891378,reads_SRR23891363,reads_SRR23891366,reads_SRR23891367,reads_SRR23891370,reads_SRR23891372
3986,C5_01745W,863,827,465,529,668,1021,580,720,861,431,283,533,57,184,239,778
3987,C5_01755C,35,22,2,2,29,58,36,12,7,1,17,95,0,1,4,36
3988,C5_01765C,1521,1228,856,1089,1098,1999,1365,1055,1255,1035,738,1204,1719,424,243,1608
3989,C5_01775C,947,474,621,182,914,1058,703,833,886,835,364,386,597,193,289,1154
3990,C5_01785W,343,211,182,14,242,518,353,345,222,113,155,217,219,30,69,334


Now we will concatinate the diff1 and diff2 with the counts_sum1.csv

In [34]:
final= pd.read_csv("no_length/reads_gene/counts_sum1.csv", sep="\t")
diff2= pd.read_csv("no_length/reads_gene/diff2.csv", sep="\t")
diff1= pd.read_csv("no_length/reads_gene/diff1.csv", sep="\t")
dfs=[]
dfs.append(final)
dfs.append(diff1)
dfs.append(diff2)

final_counts_sum= pd.concat(dfs, axis=0)   # axis 0 means concatinating along y axis or vertically

In [35]:
final_counts_sum

,Geneid,reads_SRR23891364,reads_SRR23891365,reads_SRR23891368,reads_SRR23891369,reads_SRR23891371,reads_SRR23891373,reads_SRR23891374,reads_SRR23891375,reads_SRR23891376,reads_SRR23891377,reads_SRR23891378,reads_SRR23891363,reads_SRR23891366,reads_SRR23891367,reads_SRR23891370,reads_SRR23891372
0,C1_00010W,0,0,81,2,46,0,14,48,42,7,8,8,0,42,3,13
1,C1_00020C,0,0,12,0,0,1,22,0,1,2,1,0,0,31,0,29
2,C1_00030C,0,0,0,0,0,0,0,0,1,3,1,0,0,2,0,0
3,C1_00040W,67,113,40,41,67,75,26,5,5,43,3,15,0,6,18,54
4,C1_00050C,0,0,5,0,1,3,0,4,3,0,0,60,260,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,C5_01745W,863,827,465,529,668,1021,580,720,861,431,283,533,57,184,239,778
1,C5_01755C,35,22,2,2,29,58,36,12,7,1,17,95,0,1,4,36
2,C5_01765C,1521,1228,856,1089,1098,1999,1365,1055,1255,1035,738,1204,1719,424,243,1608
3,C5_01775C,947,474,621,182,914,1058,703,833,886,835,364,386,597,193,289,1154


In [36]:
final_counts_sum.to_csv("no_length/reads_gene/final_counts_sum.csv", sep="\t", index=False)

Remvoing unneccesary csv files

In [37]:
!rm no_length/reads_gene/counts_sum1.csv
!rm no_length/reads_gene/counts_sum.csv
!rm no_length/reads_gene/diff1.csv
!rm no_length/reads_gene/diff2.csv

Now we finaaly have our reads per gne counts with us. With this we can now perform DeSeq and TE

In [23]:
## This code is for creating the final Length file which will have length of all the gneids which we can further use for the deseq and te anysis
len_dfs=[]

lenA= pd.read_csv("no_length/reads_gene/length_A.csv", sep='\t')
lenB= pd.read_csv("no_length/reads_gene/length_B.csv", sep='\t')
len_noAB=pd.read_csv("no_length/reads_gene/length_noAB.csv", sep='\t')

len_dfs.append(lenA)
len_dfs.append(lenB)
len_dfs.append(len_noAB)

final_len= pd.concat(len_dfs, axis=0)
final_len=final_len.drop_duplicates(subset=["Geneid"])
final_len.to_csv("no_length/reads_gene/final_len.csv", sep="\t", index=False)


